# Penetration Testing

## Before You Begin
- Always obtain written permission before testing any system you do not own.
- Run these scripts only against lab targets or public test ranges (e.g., Hack The Box).
- Document what you test, why, and which safeguards you used; that narrative is important evidence for the portfolio.

## 1. Passive Reconnaissance
Start with information that is already public. Here we combine DNS resolution with a geolocation API to profile owned infrastructure.

In [3]:
import socket
import requests

def get_domain_info(domain):
    try:
        # Get IP address (active but low-risk)
        ip = socket.gethostbyname(domain)
        print(f"IP Address: {ip}")
        # Get public WHOIS-like info (passive, using a free API)
        response = requests.get(f"https://ipapi.co/{ip}/json/")
        if response.status_code == 200:
            data = response.json()
            print(f"Organization: {data.get('org', 'Unknown')}")
            print(f"City: {data.get('city', 'Unknown')}")
            print(f"Country: {data.get('country_name', 'Unknown')}")
        else:
            print("Could not fetch WHOIS data.")
    except Exception as e:
        print(f"Error: {e}")
# Example: Use a public domain (never use without permission!)
get_domain_info("python.com")

IP Address: 185.158.133.1
Organization: CLOUDFLARENET
City: Frankfurt am Main
Country: Germany
Organization: CLOUDFLARENET
City: Frankfurt am Main
Country: Germany


## 2. Black-Box Recon

In [4]:
def black_box_recon(url: str, timeout: float = 5.0) -> Dict[str, Any]:
    try:
        response = requests.head(url)
        print("Black Box Findings:")
        print(f"Server: {response.headers.get('Server', 'Unknown')}")
        print(f"Content-Type: {response.headers.get('Content-Type',
        'Unknown')}")
    except Exception as e:
        print(f"Error: {e}")
        
url = "http://python.com"
known_info = {"server": "Apache 2.4", "vulns": "Check CVE-2021-1234"}
black_box_recon(url)

Black Box Findings:
Server: cloudflare
Content-Type: Unknown


## 3. Port Scanning (Socket-Level)

In [5]:
import socket

def scan_ports(host, ports):
    open_ports = []
    for port in ports:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1)
        result = sock.connect_ex((host, port))
        if result == 0:
            open_ports.append(port)
        sock.close()
    return open_ports

host = "127.0.0.1"
ports = [80, 443, 22, 8080]
open_ports = scan_ports(host, ports)
print(f"Open ports on {host}: {open_ports}")

Open ports on 127.0.0.1: [22]


## 4. Scanning with `python-nmap`

In [7]:
import nmap

def nmap_scan(host, port_range='1-1024'):
    nm = nmap.PortScanner()
    try:
        nm.scan(host, port_range, arguments='-sV') # -sV for service version detection
        for host in nm.all_hosts():
            print(f"Host: {host} ({nm[host].hostname()})")
            print(f"State: {nm[host].state()}")
            for proto in nm[host].all_protocols():
                print(f"Protocol: {proto}")
                lport = nm[host][proto].keys()
                for port in sorted(lport):
                    service = nm[host][proto][port]
                    print(f"Port: {port}\tState: {service['state']}\tService:    {service.get('name', 'unknown')} {service.get('version', '')}")
    except Exception as e:
        print(f"Error: {e}")
# Example: Scan localhost
nmap_scan('127.0.0.1', '1-10')

PortScannerError: 'nmap program was not found in path. PATH is : c:\\Users\\Micha\\miniconda3\\envs\\dp4ai;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai\\Library\\mingw-w64\\bin;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai\\Library\\usr\\bin;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai\\Library\\bin;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai\\Scripts;C:\\Users\\Micha\\miniconda3\\envs\\dp4ai\\bin;C:\\Users\\Micha\\miniconda3\\condabin;C:\\Program Files\\Eclipse Adoptium\\jdk-21.0.5.11-hotspot\\bin;C:\\Windows\\System32\\HWAudioDriverLibs;C:\\Windows\\system32;C:\\Windows;C:\\Windows\\System32\\Wbem;C:\\Windows\\System32\\WindowsPowerShell\\v1.0;C:\\Windows\\System32\\OpenSSH;C:\\Users\\Administrator\\AppData\\Local\\Microsoft\\WindowsApps;C:\\WINDOWS\\system32;C:\\WINDOWS;C:\\WINDOWS\\System32\\Wbem;C:\\WINDOWS\\System32\\WindowsPowerShell\\v1.0;C:\\WINDOWS\\System32\\OpenSSH;C:\\Program Files\\Git\\cmd;C:\\Program Files\\dotnet;C:\\Program Files\\nodejs;

## Extending the Notebook
- Correlate scan results with CVE feeds (e.g., NVD API) to prioritise findings.
- Add a simple report generator (Markdown or PDF) that captures run parameters and outputs.
- Compare passive vs active intel by mapping domains, subdomains, and discovered services.
- Instrument basic logging so you can evidence methodology and chain-of-custody.

> Document what you tested, why it mattered, the results, and how the evidence supports learning outcomes. That story is as important as the tooling.